In [ ]:
# Remove sinusoidal noise from image

# Steps
# Step 1: Read the image
# Step 2: Compute the Fourier Transform of the image
# Step 3: Get the centered Fourier Transform spectrum and display
# Step 4: Spot the periodic noise and pattern in the FT image
# Step 5: Block the periodic noise
# Step 6: Convert the image back to the spatial domain from the frequency domain
# Step 7: Compute the inverse transform of the image
# Step 8: Display

# This code ran on Google collab

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage import data, img_as_float

import matplotlib.image as pltImg

from google.colab.patches import cv2_imshow as imshow

from google.colab import drive
drive.mount('/content/drive')


sinusoidalImagePath = '/content/drive/MyDrive/sinusoidalImage.png'
imageFiltered = '/content/drive/MyDrive/filteredImage.png'

def readImage(imageUrl):
  img = cv.imread(imageUrl, 0)
  return img

def showImage(img):
  imshow(img)

def computeFourierTransform(img):
  return np.fft.fft2(img)

def computerCenteredFFT(fftImg):
  return np.fft.fftshift(fftImg)

def plotMagnitueSpectrum(magnitude_spectrum):
  plt.subplots(figsize=(30,30))
  plt.imshow(magnitude_spectrum, cmap = 'gray')
  plt.title('Magnitude Spectrum')
  plt.xticks([])
  plt.yticks([])
  plt.show()  

def medianBlur(img, kernelSize=3):
  rows, cols = sinusoidalImage.shape
  crow,ccol = math.ceil(rows/2) , math.ceil(cols/2)
  final = img[:]

  for y in range(len(img)):
      for x in range(y):
          final[y,x]=img[y,x]

  rear = kernelSize // 2
  for y in range(rear,img.shape[0]-1-rear):
      for x in range(rear,img.shape[1]-rear-1):
          # ignoring [0,0] point
          if [x, y] == [crow, ccol] or \
            [x, y] == [crow+1, ccol] or \
            [x, y] == [crow-1, ccol] or \
            [x, y] == [crow, ccol-1] or \
            [x, y] == [crow, ccol+1] or \
            [x, y] == [crow-1, ccol-1] or \
            [x, y] == [crow-1, ccol+1] or \
            [x, y] == [crow+1, ccol-1] or \
            [x, y] == [crow+1, ccol+1]:
            continue
          members = []
          for padding in range(1, kernelSize // 2 + 1):
            members.append(img[y-padding,x-padding])
            members.append(img[y,x-padding])
            members.append(img[y+padding,x-padding])
            members.append(img[y-padding,x])
            members.append(img[y+padding,x])
            members.append(img[y-padding,x+padding])
            members.append(img[y,x+padding])
            members.append(img[y+padding,x+padding])
          members.append(img[y,x])
          members.sort()
          final[y,x]=members[len(members) // 2]
  return final



def removeSinusoidalNoise(sinusoidalImage):
  # Transform original image using FFT
  fftImage = computeFourierTransform(sinusoidalImage)

  # Centering information
  centeredFftImage = computerCenteredFFT(fftImage)

  # Find spectrum of the FFT representation
  spectrum = 20*np.log(np.abs(centeredFftImage))

  # Applying filter to smooth the spectrum before finding peaks
  spectrumBlurred = medianBlur(spectrum, 3)
  imAsFloat = img_as_float(spectrumBlurred)

  # Finding peaks
  coordinates = peak_local_max(imAsFloat, min_distance=4)

  # Get the center of image
  rows, cols = sinusoidalImage.shape
  crow,ccol = math.ceil(rows/2) , math.ceil(cols/2)

  # Applying a small cross mask to selected peak points
  for pair in coordinates:
    x = pair[0]
    y = pair[1]
    if [x, y] == [crow, ccol] or \
        [x, y] == [crow+1, ccol] or \
        [x, y] == [crow-1, ccol] or \
        [x, y] == [crow, ccol-1] or \
        [x, y] == [crow, ccol+1] or \
        [x, y] == [crow-1, ccol-1] or \
        [x, y] == [crow-1, ccol+1] or \
        [x, y] == [crow+1, ccol-1] or \
        [x, y] == [crow+1, ccol+1]:
      continue
    centeredFftImage[x, y] = 0
    centeredFftImage[x, y-1] = 0
    centeredFftImage[x, y+1] = 0
    centeredFftImage[x-1, y] = 0
    centeredFftImage[x+1, y] = 0

  # Get back the spectrum
  tunnedSpectrum = 20*np.log(np.abs(centeredFftImage))

  # Perform the iFFT
  f_ishift = np.fft.ifftshift(centeredFftImage)
  img_back = np.fft.ifft2(f_ishift)

  # Get the abs of iFFT
  img_back = np.abs(img_back)

  return img_back

sinusoidalImage = readImage(sinusoidalImagePath)
img_back = removeSinusoidalNoise(sinusoidalImage)

# increase contrast: X' = aX+b, a for contrast and 0 for brightness ; alpha = contrast/127+1, beta=brightness - contrast)
# using cv for demonstration only
adjusted = cv.convertScaleAbs(img_back, alpha=1.5, beta=-63)
showImage(adjusted)